# Extraccion de datos

In [5]:
import psycopg2
import pandas as pd
from query_sql import query_POSTGRESQL 
import os, sys

#%% RUTAS

path_files = os.getcwd()
# path_sql = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir), "funciones_sql") 
path_sql = os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir), "Tarea2") 


#%% IMPORT DE FUNCIONES

sys.path.insert(0, path_sql)
from query_sql import query_POSTGRESQL, DataFrame2DataBase

#%% EXTRACCION DE DATOS

query_sql = '''SELECT * FROM dtes'''
data = query_POSTGRESQL(path_sql, 'USM', query_sql)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            535 non-null    int64 
 1   rutemisor     535 non-null    object
 2   folio         535 non-null    int64 
 3   date          535 non-null    object
 4   xml_dte       535 non-null    object
 5   convert_from  535 non-null    object
dtypes: int64(2), object(4)
memory usage: 25.2+ KB


# Transformacion de datos

In [9]:
import xml.etree.ElementTree as ET

df_new = pd.DataFrame()
rows = []

for xml in data['convert_from'].head(20):
    # print(xml)

    root = ET.fromstring(xml)

    # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    rut_emisor = root.find('.//default:RutEmisor', namespace).text

    razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

    NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

    # a veces DscItem no existe
    DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

    if DscItem is not None: 
        nuevas_filas = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
            "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text
    }    
    else: # SI DscItem NO TIENE NADA 
        nuevas_filas = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
            "DscItem": 'NO HAY',
        }

    rows.append(nuevas_filas)


In [28]:
# extraer la glosa

import xml.etree.ElementTree as ET

df_new = pd.DataFrame()
rows = []

for xml in data['convert_from']:
    # print(xml)

    root = ET.fromstring(xml)

    # # Accede a la información de una etiqueta específica
    # etiqueta_deseada = root.find('.//etiqueta')

    # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    # Encuentra y extrae el valor de la etiqueta RutEmisor
    rut_emisor = root.find('.//default:RutEmisor', namespace).text

    razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

    # detalle = root.find('.//{http://www.sii.cl/SiiDte}Detalle').text
    NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

    # a veces DscItem no existe
    DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

    if DscItem is not None:
        # print('esta la etiqueta')
        nuevas_filas = {
        "rut_emisor": rut_emisor,
        "razon_social": razon_social,
        "NmbItem": NmbItem,
        "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text
        
    }
    else:
    # La etiqueta no fue encontrada, manejar la situación en consecuencia
        # print("La etiqueta DscItem no fue encontrada.")
        nuevas_filas = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
            "DscItem": 'NO HAY',
        }

    rows.append(nuevas_filas)

df_new = pd.DataFrame(rows)

In [3]:
df_new.info()

NameError: name 'df_new' is not defined

In [2]:
df_new['DscItem'].value_counts()

NameError: name 'df_new' is not defined

In [19]:
def concat_if_not_no_hay(df):
    # Filtrar filas donde DscItem no sea 'NO HAY'
    filtered_df = df[df['DscItem'] != 'NO HAY'].copy()  # Añadir .copy() para evitar la advertencia
    
    # Concatenar NmbItem y DscItem
    filtered_df.loc[:, 'Concatenated'] = filtered_df['NmbItem'] + ' ' + filtered_df['DscItem']  # Usar .loc[]
    
    return filtered_df

In [31]:
concat_if_not_no_hay(df_new).info()

<class 'pandas.core.frame.DataFrame'>
Index: 382 entries, 0 to 534
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rut_emisor    382 non-null    object
 1   razon_social  382 non-null    object
 2   NmbItem       382 non-null    object
 3   DscItem       380 non-null    object
 4   Concatenated  380 non-null    object
dtypes: object(5)
memory usage: 17.9+ KB
